In [2]:
import dotenv
dotenv.load_dotenv('../.secrets', override=True)
!pip install rc_sqlalchemy_cfg 

In [3]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from rc_sqlalchemy_cfg.sqlalchemy_cfg import configure_base_model

def get_session():
    engine = create_engine(os.getenv('ENSEMBLE_DB_URI'))
    session_maker = sessionmaker(autocommit=False, autoflush=True, bind=engine)
    session = scoped_session(session_maker)
    configure_base_model(session)
    return session

ens_session = get_session()

In [4]:
import re
from rc_sqlalchemy_cfg.models import InvestmentEntityDocument, InvestmentEntity

def get_doc_to_product():
    ens_session = get_session()
    
    doc_product = {int(re.search(r'"ID":\s*(\d+)', d.object_version).group(1)): d.product_id for d in 
            ens_session.query(InvestmentEntity.product_id, InvestmentEntityDocument.object_version).join(InvestmentEntityDocument).all()}
    return doc_product
    
doc_to_product = get_doc_to_product()
doc_to_product

{85313: 1,
 47405: 1,
 47407: 2,
 79950: 3,
 79956: 3,
 46517: 4,
 122853: 4,
 109969: 5,
 41828: 5,
 41840: 5,
 41823: 5,
 41824: 5,
 84396: 6,
 54831: 6,
 114077: 7,
 78529: 7,
 80055: 8,
 46519: 9,
 121894: 9,
 47409: 9,
 41873: 10,
 115715: 10,
 90031: 10,
 41889: 10,
 111343: 11,
 83554: 11,
 119105: 11,
 45318: 11,
 112369: 12,
 104146: 12,
 45319: 12,
 44060: 13,
 89656: 13,
 91285: 14,
 78530: 14,
 78531: 15,
 91286: 15,
 114078: 16,
 86749: 16,
 86438: 16,
 43198: 17,
 43199: 17,
 113533: 17,
 101931: 17,
 42599: 18,
 52071: 19,
 69625: 19,
 51615: 19,
 45514: 19,
 47650: 19,
 46926: 19,
 114167: 20,
 47413: 20,
 114542: 21,
 47414: 21,
 112959: 21,
 86439: 21,
 77753: 6840,
 65300: 6840,
 53840: 6840,
 116711: 6840,
 81855: 6840,
 81926: 6840,
 81856: 6840,
 116809: 6840,
 122609: 22,
 116704: 22,
 45322: 22,
 47415: 22,
 113590: 23,
 113581: 23,
 85539: 23,
 45545: 23,
 122836: 23,
 113587: 23,
 112370: 24,
 112308: 24,
 111351: 24,
 78535: 26,
 53674: 28,
 111286: 28,
 1054

In [9]:
import pandas as pd
import pyodbc


def get_document_dates(doc_to_product, num_months=1):
    conn = pyodbc.connect("DSN=Azure Databricks", autocommit=True)
    df = conn.execute(
        "select document_mf_id, document_type, document_date from fund_document_v3 "
        "where document_type in ('quarterly_report', 'monthly_report')").fetchall()
    
    df = pd.DataFrame([list(d) for d in df],
                            columns=['document_mf_id', 'document_type', 'document_date'])
    df['start_date'] = pd.to_datetime(df['document_date']) + pd.Timedelta(days=1)
    df['end_date'] = df['start_date'] + pd.DateOffset(months=num_months) - pd.Timedelta(days=1)
    
    df = df.drop(columns=['document_date', 'document_type'])
    df['product_id'] = df['document_mf_id'].map(doc_to_product)
    return df.dropna()

doc_dates = get_document_dates(doc_to_product, num_months=1)
doc_dates

,document_mf_id,start_date,end_date,product_id
0,123952,2024-04-01 00:00:00+00:00,2024-04-30 00:00:00+00:00,486.0
1,123951,2024-04-01 00:00:00+00:00,2024-04-30 00:00:00+00:00,6771.0
2,123779,2024-04-01 00:00:00+00:00,2024-04-30 00:00:00+00:00,203.0
4,123731,2024-04-01 00:00:00+00:00,2024-04-30 00:00:00+00:00,281.0
5,123678,2024-04-01 00:00:00+00:00,2024-04-30 00:00:00+00:00,268.0
...,...,...,...,...
734,89742,2022-10-01 00:00:00+00:00,2022-10-31 00:00:00+00:00,1029.0
735,101535,2023-04-01 00:00:00+00:00,2023-04-30 00:00:00+00:00,672.0
736,101536,2023-04-01 00:00:00+00:00,2023-04-30 00:00:00+00:00,674.0
737,101539,2023-04-01 00:00:00+00:00,2023-04-30 00:00:00+00:00,995.0


In [6]:
from sqlalchemy import func
from rc_sqlalchemy_cfg.models import ProductReturn


def query_returns(queries):
    """
    queries: A list of tuples, each containing (parent_id, start_date, end_date)
    Returns a list of tuples with the average return for each condition
    """
    ens_session = get_session()
    results = []
    for i, row in queries.iterrows():
        avg_return = ens_session.query(func.avg(ProductReturn.value)).filter(
            ProductReturn.product_id == row['product_id'],
            ProductReturn.value_date >= row['start_date'],
            ProductReturn.value_date <= row['end_date']
        ).scalar()
        results.append((row['document_mf_id'], avg_return))
    return results
doc_returns = pd.DataFrame(query_returns(doc_dates), columns=['document_mf_id', 'avg_return']).dropna()

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x000002222275C4A0>
Traceback (most recent call last):
  File "C:\Users\viksu\AppData\Local\Programs\Python\Python311\Lib\weakref.py", line 370, in remove
    self = selfref()
           ^^^^^^^^^
KeyboardInterrupt: 

KeyboardInterrupt



In [13]:
def get_doc_returns(num_months=1):
    doc_to_product = get_doc_to_product()
    doc_dates = get_document_dates(doc_to_product, num_months)
    doc_returns = pd.DataFrame(query_returns(doc_dates), columns=['document_mf_id', 'avg_return']).dropna()
    doc_returns.set_index('document_mf_id', inplace=True)
    doc_returns['avg_return'] = doc_returns['avg_return'].astype(float)
    return doc_returns

doc_returns = get_doc_returns(num_months=1)
doc_returns

,avg_return
document_mf_id,
123952,1.0000
123779,-0.0009
123731,2.6349
123678,0.1000
123676,0.3317
...,...
89742,-0.0700
101535,1.7600
101536,3.4600


In [14]:
# pickle result
import pickle
with open('doc_returns.pkl', 'wb') as f:
    pickle.dump(doc_returns, f)

In [ ]:
# USE PROD